In [ ]:
from awe.data import swde, qa_dataset

In [ ]:
sds = swde.Dataset(suffix='-exact')
websites = sds.verticals[0].websites

In [ ]:
page = websites[0].pages[0]
page.file_path

In [ ]:
import selectolax

In [ ]:
tree = selectolax.parser.HTMLParser(page.html)

In [ ]:
tree.strip_tags([
    'script',
    'style',
    'head',
    '[document]',
    'noscript',
    'iframe'
])

In [ ]:
nodes_text = qa_dataset.collapse_whitespace(' '.join(qa_dataset.collapse_whitespace(n.text(deep=False)) for n in tree.body.traverse()))
body_text = qa_dataset.collapse_whitespace(tree.body.text(separator=' '))
len(nodes_text), len(body_text)

In [ ]:
import pandas as pd
from awe import selectolax_utils

In [ ]:
df = pd.DataFrame([
    {
        'xpath': selectolax_utils.get_xpath(node),
        'text': text
    }
    for node in tree.body.traverse(include_text=True)
    if selectolax_utils.is_text(node) and not (text := qa_dataset.collapse_whitespace(node.text(deep=False))).isspace() and text != ''
])

In [ ]:
df